<a href="https://colab.research.google.com/github/Dhanyabahadur/ML/blob/master/recommendation%20system%20e%20commerce%20%20%20%20question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import requests

Recommendation systems are used to understand customer behaviour and recommend products.As the given dataset consists of enough data of our customers and their preferred choices , so I am using **content-based approach**.

Importing libraries 

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


*As I am importing dataset from drive to colab , so I am importing necessary libraries like **pydrive** and **os** .*


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# **Data Preparation**

In [0]:
# ecommerce_sample_dataset.csv
download = drive.CreateFile({'id': '17na1xGx0fRCpjyMqdBLcuYHCh1pHZHa3'})

In [0]:
# ecommerce_sample_dataset.csv
folder_id = '17na1xGx0fRCpjyMqdBLcuYHCh1pHZHa3'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()
for item in lister:
    print(item['title'])
    # this should tell you which mimetype the file you're trying to download 
    # has. 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {



        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',

        # Drive document as folder
        'application/vnd.google-apps.folder': 'application/folder',

        #Drive document as image
        'application/vnd.google-apps.photo': 'application/png',

    # see https://developers.google.com/drive/v3/web/mime-types
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[item['mimeType']]
        item.GetContentFile(item['title'], mimetype=download_mimetype)

        #item.GetContentFile(item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(item['title'])

*Downloaded the ecommerce_sample_dataset.csv*  

In [0]:
download.GetContentFile('ecommerce_sample_dataset.csv')


As the given datset have lot of features like time interval , product's url , product name, product category , price(retail and discounted), image of product, rating and brand.


**Load the raw data into pandas dataframe and separate into columns**. 

Consider columns like crawl_timestamp, product_url,product_name,reatail_price, discounted_price,description, product_rating, overall_rating and brand.

In [7]:
df = pd.read_csv("ecommerce_sample_dataset.csv")
df_new=df.iloc[:,[1,2,3,6,7,10,11,12,13]]
df_new['index'] = df_new.index
df_new.shape

(20000, 10)

**Verify that the data is loaded correctly**

In [8]:
df_new.head()

,crawl_timestamp,product_url,product_name,retail_price,discounted_price,description,product_rating,overall_rating,brand,index
0,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,999.0,379.0,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,0
1,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,1
2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,999.0,499.0,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,2
3,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,699.0,267.0,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,3
4,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,4


In the given dataset there are various datapoints which have null value . So consider frequency of those null points in dataset . This will help us decide our course of action like whether to delete those rows or put mean value of the column in those datapoints.

In [9]:
null_counts = df_new.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
crawl_timestamp        0
product_url            0
product_name           0
retail_price          78
discounted_price      78
description            2
product_rating         0
overall_rating         0
brand               5864
index                  0
dtype: int64


We know the shape of the dataframe is (20000,10) before altering it.

In [10]:
print("Data types and their frequency\n{}".format(df_new.dtypes.value_counts()))

Data types and their frequency
object     7
float64    2
int64      1
dtype: int64


In [11]:
object_columns_df = df_new.select_dtypes(include= ['object'])
print(object_columns_df.iloc[0])

crawl_timestamp                            2016-03-25 22:59:23 +0000
product_url        http://www.flipkart.com/alisha-solid-women-s-c...
product_name                     Alisha Solid Women's Cycling Shorts
description        Key Features of Alisha Solid Women's Cycling S...
product_rating                                   No rating available
overall_rating                                   No rating available
brand                                                         Alisha
Name: 0, dtype: object


Create price buckets to categorize products based on their price.

In [0]:
new_price = df_new["discounted_price"]
df_new["new_price"] = new_price[1]
bins = [0,500,1000,2000,3000,4000,5000,10000,20000,40000,60000]
labels = ["<500","500-1000","1000-2000","2000-3000","3000-4000","4000-5000","5000-10000","10000-20000","20000-40000",">40000"]
df_new['price_bracket'] = pd.cut(df_new['new_price'], bins, labels=labels)

*Consider only those columns which are required for the recommendation system and remove all the NA values.*

In [0]:
features = [ "product_url","product_name", "discounted_price","description","brand"]

for feature in features:
  df_new[feature] = df_new[feature].fillna('')

*A large string named combined_features is created to combine all columns into a large string*

In [14]:
def combine_features(row):
  try:
    return row['product_url']+" "+ row['product_name']+" "+  row['discounted_price']+" "+ row['description']+" "+row['brand']
  except:
    print("Error:", row)

df_new["combined_features"]= df_new.apply(combine_features, axis = 1)

Streaming output truncated to the last 5000 lines.
product_rating                                    No rating available
overall_rating                                    No rating available
brand                                                                
index                                                           19614
new_price                                                       22646
price_bracket                                             20000-40000
Name: 19614, dtype: object
Error: crawl_timestamp                             2015-12-01 10:15:43 +0000
product_url         http://www.flipkart.com/hugo-chavez-women-s-sh...
product_name                         Hugo Chavez Women's Sheath Dress
retail_price                                                     2499
discounted_price                                                  997
description         Hugo Chavez Women's Sheath Dress - Buy Multico...
product_rating                                                      3
overa

*In order to make the recommendation system accurate remove stopwords like an, in the, on etc.*

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
df_new['combined_features'] = df_new['combined_features'].astype(str).str.lower().str.split()
df_new["features"]=df_new["combined_features"].apply(lambda x: [word for word in x if word not in stop_words])
df_new["features"]=df_new["features"].apply(lambda x: " ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


*I want to find count matrix, this matrix is nothing but number of occurences of each word in each feature.To find count matrix I am using "CountVectorizer()" function .*

In [0]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df_new["features"])

*Using "cosine_similarity()" function to find Cosine similarity between these count_matrix . *

In [0]:
cosine_sim = cosine_similarity(count_matrix)

I have created 4 functions that will give product name, brand and product URL from product index and vis a vis .

In [0]:
def get_title_from_index(index):
    return df_new[df_new.index == index]["product_name"].values[0]

def get_home(index):
    return df_new[df_new.index == index]["product_url"].values[0]

def get_home(index):
    return df_new[df_new.index == index]["brand"].values[0]

def get_index_from_title(title):
    return df_new[df_new.product_name == title]["index"].values[0]

Taking user input for the product name .

In [0]:
product_user_likes = 'AW Bellies'

Now we will extract the index of the product and create a list of similiar products and sort based on high cosine similarity value .

In [0]:
product_index = get_index_from_title(product_user_likes)

similar_products =  list(enumerate(cosine_sim[product_index]))

sorted_similar_products = sorted(similar_products,key=lambda x:x[1],reverse=True)

Below is the product name and product url of first 5 similiar products

In [21]:
i=0
for element in sorted_similar_products:
  print(get_title_from_index(element[0]),get_home(element[0]))
  i = i+1
  if i > 5:
    break
    

Alisha Solid Women's Cycling Shorts Alisha
FabHomeDecor Fabric Double Sofa Bed FabHomeDecor
AW Bellies AW
Alisha Solid Women's Cycling Shorts Alisha
Sicons All Purpose Arnica Dog Shampoo Sicons
Eternal Gandhi Super Series Crystal Paper Weights  with Silver Finish Eternal Gandhi
